In [1]:
import h5py
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from tqdm import tqdm_notebook as tqdm
import torch.utils.data as utils
from torch.autograd import Variable
from torchvision import transforms
from torch import nn, optim
import torch.nn.functional as F

In [ ]:
df_train = h5py.File("./data/X_train.h5", "r")
x_train = df_train["features"][:183143]
x_test  = df_train["features"][183143:]

In [22]:
y_train = pd.read_csv("./data/y_train.csv").values[:183143, 1].squeeze()
y_test = pd.read_csv("./data/y_train.csv").values[183143:, 1].squeeze()

In [29]:
data = np.load('./data/train.npy')
x_eval = np.load('./data/test.npy')

In [4]:
x_train, x_test = data[:183143], data[183143:]

In [5]:
def get_waves(x_train):
    freq  = 125*np.fft.fftfreq(x_train.shape[1])
    eeg   = np.abs(np.fft.fft(x_train))
    delta = np.sum(eeg[:, (freq > 0.5) & (freq <= 4)], axis = 1)
    theta = np.sum(eeg[:, (freq > 4) & (freq <= 7)], axis = 1)
    alpha = np.sum(eeg[:, (freq > 7) & (freq <= 13)], axis = 1)
    beta  = np.sum(eeg[:, (freq > 13) & (freq <= 30)], axis = 1)
    gamma = np.sum(eeg[:, (freq > 30)], axis = 1)
    return np.array([delta, theta, alpha, beta, gamma])

In [6]:
y_train = torch.tensor(y_train).long()
y_test  = torch.tensor(y_test).long()

In [30]:
x_train = torch.tensor(x_train).float()
x_test  = torch.tensor(x_test).float()
x_eval  = torch.tensor(x_eval).float()

C:\Users\Utilisateur\.conda\envs\keelab\lib\site-packages\ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
C:\Users\Utilisateur\.conda\envs\keelab\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [8]:
test_mean  = x_test.mean(dim = 0 ,keepdim=True)
test_std   = x_test.std(dim = 0 ,keepdim=True)
train_mean = x_train.mean(dim = 0 ,keepdim=True)
train_std  = x_train.std(dim = 0 ,keepdim=True)

In [32]:
eval_mean = x_eval.mean(dim = 0 ,keepdim=True)
eval_std = x_eval.std(dim = 0 ,keepdim=True)
x_eval  = (x_eval - eval_mean) / eval_std

In [9]:
x_test  = (x_test - test_mean) / test_std
x_train = (x_train - train_mean) / train_std


In [10]:
train_data = utils.TensorDataset(x_train, y_train)
test_data  = utils.TensorDataset(x_test, y_test)

In [11]:
train_loader = utils.DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = utils.DataLoader(test_data, batch_size=64, shuffle=True)

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(16, 128)
        self.fc2 = nn.Linear(128, 64)
        self.do1 = nn.Dropout(p = 0.3)
        self.fc3 = nn.Linear(64, 16)
        self.do2 = nn.Dropout(p = 0.25)
        self.fc4 = nn.Linear(16, 3)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.do1(x)
        x = F.relu(self.fc3(x))
        x = self.do2(x)
        x = self.fc4(x)
        return F.log_softmax(x, dim=1)

In [13]:
loss_function = nn.NLLLoss()

In [14]:
model = Net()

In [15]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [16]:
epochs = 50
for e in tqdm(range(epochs)):
    running_loss = 0
    for data, labels in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(train_loader)))

Epoch 0 - Training loss: 1.000247526676983
Epoch 1 - Training loss: 0.991282894259812
Epoch 2 - Training loss: 0.9898961142882194
Epoch 3 - Training loss: 0.9887845430817328
Epoch 4 - Training loss: 0.9879042007013437
Epoch 5 - Training loss: 0.9875378128867479
Epoch 6 - Training loss: 0.9871817643329199
Epoch 7 - Training loss: 0.9870561729312193
Epoch 8 - Training loss: 0.9867503568686279
Epoch 9 - Training loss: 0.9861671906472252
Epoch 10 - Training loss: 0.9856953277694521
Epoch 11 - Training loss: 0.9851156873856284
Epoch 12 - Training loss: 0.9854971408677384
Epoch 13 - Training loss: 0.9846565163610367
Epoch 14 - Training loss: 0.9851601572823141
Epoch 15 - Training loss: 0.984463913094156
Epoch 16 - Training loss: 0.9841398324981425
Epoch 17 - Training loss: 0.984217988786624
Epoch 18 - Training loss: 0.9842312471338455
Epoch 19 - Training loss: 0.9837566175550785
Epoch 20 - Training loss: 0.9839033638989484
Epoch 21 - Training loss: 0.9838327181705805
Epoch 22 - Training loss

In [18]:
torch.save(model.state_dict(), 'First')

In [33]:
y_pred = torch.exp(model(x_eval))

In [34]:
test = np.argmax(y_pred.detach().numpy(), axis = 1)

In [33]:
np.sum(test == y_test)/len(y_test)

0.5114726529156209

In [27]:
np.sum(test == y_test)/len(y_test)

0.5131034131301678

In [24]:
model.eval()

Net(
  (fc1): Linear(in_features=16, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (do1): Dropout(p=0.3)
  (fc3): Linear(in_features=64, out_features=16, bias=True)
  (do2): Dropout(p=0.25)
  (fc4): Linear(in_features=16, out_features=3, bias=True)
)

In [36]:
df = pd.DataFrame()
df['id'] = range(len(test))
df['label'] = test
df.to_csv('./data/result.csv')